In [1]:
# Python ≥3.5 is required
import sys

assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

from sklearn.impute import SimpleImputer
#from sklearn.pipeline import Pipelines
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# TensorFlow ≥2.0 is required
import tensorflow_addons as tfa
import tensorflow as tf
assert tf.__version__ >= '2.0'

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

from keras.models import Sequential
#from keras.layers.wrappers import TimeDistributed
#from keras.layers import Dense,LSTM,Conv2D, BatchNormalization,Flatten, MaxPooling2D
#from keras.layers import Conv2DTranspose,Concatenate,UpSampling2D,Cropping2D
#from keras.layers import Input, Lambda, Reshape, Dropout, Activation

from tensorflow.keras.layers import Dropout, BatchNormalization, Reshape
from tensorflow.keras.layers import Dense, Conv2D, Input, MaxPooling2D, Flatten, MaxPool2D, MaxPool3D, UpSampling2D
from tensorflow.keras.layers import Conv2DTranspose, Flatten, Reshape, Cropping2D, Embedding, BatchNormalization,ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU, Activation, Input, add, multiply
from tensorflow.keras.layers import concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K

#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Common imports
import os
import glob
import numpy as np
from numpy import ones
import pandas as pd
import geopandas as gpd
import xarray as xr
import dask
import datetime
import math
import pathlib
import hashlib
dask.config.set({'array.slicing.split_large_chunks': False})

# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt


mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
# Dotenv
from dotenv import dotenv_values
# Custom utils
from utils.utils_data import *
from utils.utils_ml import *
from utils.utils_plot import *
from utils.utils_unet import *
from utils.utils_resnet import *

In [2]:
# Define paths
config = dotenv_values(".env")

In [3]:
PATH_ERA5 = config['PATH_ERA5']
PATH_EOBS = config['PATH_EOBS']

In [4]:
# Some constants
DATE_START = '1979-01-01'
DATE_END = '2021-12-31'
# train -test (80/20 of total yy), train-> split 80/20 for validation
#YY_TRAIN = [1979, 2011] # 2005-2011 for validation
#YY_VAL = [2005,2011]
#YY_TEST = [2012, 2020]
YY_TRAIN = [1979, 2015]
YY_TEST = [2016, 2021]
YY_VALID = 2005
LEVELS = [500, 925, 1000]
G = 9.80665 
LONS_INPUT = [-25, 30]
LATS_INPUT = [30, 75]
LONS_PREC = [-25, 30]
LATS_PREC = [30, 75]

PRECIP_DATA = 'ERA5' # Options: ERA5, E-OBS
PRECIP_XTRM = 0.95 # Percentile (threshold) for the extremes

In [5]:
# Load precipitation
if PRECIP_DATA == 'ERA5':
    pr = get_nc_data(PATH_ERA5 + '/precipitation/day_grid1/*nc', DATE_START, DATE_END, LONS_PREC, LATS_PREC)
    pr = pr.tp
elif PRECIP_DATA in ['E-OBS', 'EOBS']:
    pr = get_nc_data(PATH_EOBS + '/*nc', DATE_START, DATE_END, LONS_PREC, LATS_PREC)
    pr = pr.rr
    pr = pr.coarsen(lon=5, lat=5, boundary="trim").mean()
    pr = pr.fillna(0) # Over the seas. Not optimal...
else:
    raise('Precipitation data not well defined')

Extracting data for the period 1979-01-01 - 2021-12-31


In [6]:
# Define extremes
pr95 = precip_exceedance_xarray(pr, PRECIP_XTRM)

In [7]:
# Extract coordinates for precip
lats_y = pr.lat.to_numpy()
lons_y = pr.lon.to_numpy()

In [8]:
# define paths for the variables used
l_paths = ['/geopotential/day_grid1/','/temperature/day_grid1/','/relative_humidity/day_grid1/',
              '/U_wind/day_grid1/','/V_wind/day_grid1/','/total_column_water/day_grid1/']
v_vars = ['z','t2m','rh','u850','v850','tpcw']
list_vars = load_data(v_vars, l_paths, G, PATH_ERA5, DATE_START, DATE_END, LONS_INPUT, LATS_INPUT, LEVELS)
datasets = list_vars

Extracting data for the period 1979-01-01 - 2021-12-31
Extracting data for the period 1979-01-01 - 2021-12-31
Extracting data for the period 1979-01-01 - 2021-12-31
Extracting data for the period 1979-01-01 - 2021-12-31
Extracting data for the period 1979-01-01 - 2021-12-31
Extracting data for the period 1979-01-01 - 2021-12-31


In [9]:
dic = {
     'z': LEVELS,
      't': LEVELS,
      'r': LEVELS,
      'u': LEVELS,
      'v': LEVELS,
      'tcwv':None}
# Check if all have the same latitude order
for idat in range(0, len(datasets)):
    # Invert lat axis if needed
    if datasets[idat].lat[0].values < datasets[idat].lat[1].values:
        print('change lat order', idat)
        datasets[idat] = datasets[idat].reindex(lat=list(reversed(datasets[idat].lat)))

In [10]:
ds = xr.merge(datasets)

In [11]:
BATCH_SIZE=64 # try increase, decrease it

In [12]:
# Split into training and test, then I will use DataGenerator class to get the validation
ds_train = ds.sel(time=slice('{}-01-01'.format(YY_TRAIN[0]),
                             '{}-12-31'.format(YY_TRAIN[1])))
ds_test = ds.sel(time=slice('{}-01-01'.format(YY_TEST[0]),
                            '{}-12-31'.format(YY_TEST[1])))

dy_train = pr95.sel(time=slice('{}-01-01'.format(YY_TRAIN[0]),
                             '{}-12-31'.format(YY_TRAIN[1])))
dy_test = pr95.sel(time=slice('{}-01-01'.format(YY_TEST[0]),
                             '{}-12-31'.format(YY_TEST[1])))

In [13]:
dy_pr_train = pr.sel(time=slice('{}-01-01'.format(YY_TRAIN[0]),
                             '{}-12-31'.format(YY_TRAIN[1])))
dy_pr_test = pr.sel(time=slice('{}-01-01'.format(YY_TEST[0]),
                             '{}-12-31'.format(YY_TEST[1])))

In [14]:
dg_train = DataGeneratorWithExtremes(ds_train.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')),
                                     dy_pr_train.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')),
                                     dy_train.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')),
                                     dic, batch_size=BATCH_SIZE, load=True)
dg_valid = DataGeneratorWithExtremes(ds_train.sel(time=slice(f'{YY_VALID+1}', f'{YY_TRAIN[1]}')),
                                     dy_pr_train.sel(time=slice(f'{YY_VALID+1}', f'{YY_TRAIN[1]}')),
                                     dy_train.sel(time=slice(f'{YY_VALID+1}', f'{YY_TRAIN[1]}')),
                                     dic, mean=dg_train.mean, std=dg_train.std,
                                     batch_size=BATCH_SIZE, load=True)
dg_test = DataGeneratorWithExtremes(ds_test, dy_pr_test, dy_test, dic,
                                    mean=dg_train.mean, std=dg_train.std,
                                    batch_size=BATCH_SIZE, load=True, shuffle=False)

Loading data into RAM
Loading data into RAM
Loading data into RAM


In [15]:
#for now I am going to use extremes:
# Switch to precipitation extremes
dg_train.for_extremes(True)
dg_valid.for_extremes(True)
dg_test.for_extremes(True)

In [20]:
i_shape = dg_train.X.shape[1:]
o_shape = dg_train.y.shape[1:]

print(f'X shape: {i_shape}')
print(f'y shape: {o_shape}')

X shape: (46, 56, 16)
y shape: (46, 56)


In [21]:
weights = class_weight.compute_class_weight('balanced', classes = np.unique(dg_train.y_xtrm), y = np.array(dg_train.y_xtrm).flatten())

In [22]:
model_loss = weighted_binary_cross_entropy(weights = {0: weights[0].astype('float32'), 1: weights[1].astype('float32')})

In [23]:
lr = 0.0004
optimizer = tf.optimizers.Adam(learning_rate = lr)

In [24]:
### Test GANs ####

In [25]:
# distankle GAN -class
output_size = list(o_shape)
latent_dimension=64

In [26]:
def padding_block(x, factor):
    
    h, w = x.get_shape().as_list()[1:3]
    dh = 0
    dw = 0
    if h % factor > 0:
        dh = factor - h % factor
    if w % factor > 0:
        dw = factor - w % factor
    if dh > 0 or dw > 0:
        top_pad = dh//2
        bottom_pad = dh//2 + dh%2
        left_pad = dw//2
        right_pad = dw//2 + dw%2
        x = ZeroPadding2D(padding=((top_pad, bottom_pad), (left_pad, right_pad)))(x)
        
    return x

In [27]:
def get_shape_for(output_size, stride_factor):
    #output_size [46,56,1]
    next_shape = output_size.copy()
    next_shape[0] = int(np.ceil(next_shape[0]/stride_factor))
    next_shape[1] = int(np.ceil(next_shape[1]/stride_factor))

    return next_shape

In [28]:
def final_cropping_block(output_size, x):
    # Compute difference between reconstructed width and hight and the desired output size.
    h, w = x.get_shape().as_list()[1:3]
    h_tgt, w_tgt = output_size[:2]
    dh = h - h_tgt
    dw = w - w_tgt

    if dh < 0 or dw < 0:
        raise ValueError(f'Negative values in output cropping dh={dh} and dw={dw}')

    # Add to decoder cropping layer and final reshaping
    x = Cropping2D(cropping=((dh//2, dh-dh//2), (dw//2, dw-dw//2)))(x)
    #x = layers.Reshape(target_shape=self.output_size,)(x)
        
    return x

In [29]:
### Define Generator Model

random_input =layers.Input(shape = 100)

x = layers.Dense(100 * 6 * 7)(random_input)
x = layers.Activation('relu')(x)
x = layers.BatchNormalization(momentum=0.8)(x)

x = layers.Reshape((6, 7, 100))(x)

x = layers.Conv2DTranspose(filters=128, kernel_size=(2,2), strides=(2,2))(x)
x = layers.Activation('relu')(x)
x = layers.BatchNormalization(momentum=0.8)(x)

x = layers.Conv2DTranspose(filters=64, kernel_size=(2,2), strides=(2,2))(x)
x = layers.Activation('relu')(x)
x = layers.BatchNormalization(momentum=0.8)(x)

x = layers.Conv2DTranspose(filters=32, kernel_size=(2,2), strides=(2,2))(x)
x = layers.Activation('relu')(x)
x = layers.BatchNormalization(momentum=0.8)(x)


x = layers.Conv2D(filters=31, kernel_size=(1,1))(x)
x = layers.Activation('sigmoid')(x)
generated_image = final_cropping_block(output_size, x)
generator_network = Model(inputs=random_input, outputs=generated_image)
generator_network.summary()

In [30]:
def encoder_layer(input_layer, filters):
    x = layers.Conv2D(filters, kernel_size=(3,3), strides=(2,2), padding='same')(input_layer)
    x = layers.LeakyReLU(alpha=0.2)(x)
    
    return x

def decoder_layer(input_layer, skip_input, filters):
    x = layers.Conv2DTranspose(filters, kernel_size=(4,4), strides=(2,2), padding='same')(input_layer)
    x = layers.Activation('relu')(x)
    x = layers.Concatenate()([x, skip_input])
    return x

In [31]:

source_input = layers.Input(shape=i_shape)

x = padding_block(source_input, 4)
e1 = encoder_layer(x, 32)#64
e2 = encoder_layer(e1, 64)#128

bottle_neck = layers.Conv2D(64, (3,3), strides=(2,2), padding='same')(e2)
b = layers.Activation('relu')(bottle_neck)

d1 = decoder_layer(b, e2, 64)#512
d2 = decoder_layer(d1, e1, 32)#512


decoded = layers.Conv2DTranspose(16, kernel_size=(3,3), strides=(2,2), padding='same')(d2)
translated_image = layers.Activation('tanh')(decoded)
outputs = final_cropping_block(output_size, translated_image)

generator_network = Model(inputs=source_input, outputs=outputs)
print (generator_network.summary())

2022-06-29 22:33:51.591244: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-29 22:33:54.875539: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:07:00.0, compute capability: 6.1


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 46, 56, 16)  0           []                               
                                ]                                                                 
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 48, 56, 16)  0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 24, 28, 32)   4640        ['zero_padding2d[0][0]']         
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 24, 28, 32)   0           ['conv2d[0][0]']             

In [32]:
 # Discriminator
inputs = layers.Input(shape=i_shape)
x = layers.Conv2D(16, 3, padding='same')(inputs)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.SpatialDropout2D(0.3)(x) # In original: simple Dropout
x = layers.Conv2D(16, 3, padding='same')(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.SpatialDropout2D(0.2)(x) # In original: simple Dropout
x = layers.Flatten()(x)                
x = layers.Dense(16)(x) # In original: 16
x = layers.Activation('relu')(x)

next_shape = get_shape_for(output_size, stride_factor=4)
# Upsampling. In original: no decoder
x = layers.Dense(np.prod(next_shape),activation='relu')(x)
x = layers.Reshape((12,14,1))(x)
x = layers.Conv2D(16, (3,3), strides=(2, 2), padding='same', activation='relu')(x)
x = layers.Conv2DTranspose(16, (3,3), strides=(2,2), padding='same', activation='relu')(x)
x = layers.Conv2DTranspose(16, (3,3), strides=(2,2), padding='same', activation='relu')(x)
x = layers.Conv2DTranspose(16, (3,3), strides=(2,2), padding='same', activation='relu')(x)
x = layers.Conv2DTranspose(16, (3,3), strides=(2,2), padding='same', activation='relu')(x)
x = layers.Conv2D(1, (3,3), strides=(2, 2), padding='same', activation='relu')(x)
# in case we need cropping
outputs = final_cropping_block(output_size, x)

In [33]:
discriminator_network = Model(inputs=inputs, outputs=outputs)

In [34]:
discriminator_network.compile(optimizer =optimizer, loss='binary_crossentropy' , metrics=['accuracy'])

In [35]:
discriminator_network.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 46, 56, 16)]      0         
                                                                 
 conv2d_3 (Conv2D)           (None, 46, 56, 16)        2320      
                                                                 
 activation_4 (Activation)   (None, 46, 56, 16)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 28, 16)       0         
 )                                                               
                                                                 
 spatial_dropout2d (SpatialD  (None, 23, 28, 16)       0         
 ropout2D)                                                       
                                                                 
 conv2d_4 (Conv2D)           (None, 23, 28, 16)        2320

In [37]:
#Define Combined model: DCGAN
discriminator_network.trainable=False

g_output = generator_network(source_input)
d_output = discriminator_network(g_output)

dcgan_model = Model(source_input, d_output)
dcgan_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 46, 56, 16)]      0         
                                                                 
 model (Functional)          (None, 46, 56, 16)        200464    
                                                                 
 model_1 (Functional)        (None, 46, 56, 1)         56521     
                                                                 
Total params: 256,985
Trainable params: 200,464
Non-trainable params: 56,521
_________________________________________________________________


In [38]:
dcgan_model.compile(optimizer =optimizer, loss =model_loss)

In [39]:
dataset= dg_train
n_samples=dg_train.X.shape[0]
latent_dim=46*56*16
batch_size=64

In [40]:
discriminator_network.trainable=True

In [41]:
def get_predictions(input_sample, generator_network):
    input_sample = np.expand_dims(input_sample, axis=0)
    output_sample = generator_network.predict_on_batch(input_sample)
    return output_sample

def get_generated_samples(generator_network, maps_input):
    generated_samples = generator_network.predict_on_batch(maps_input)
    return generated_samples


In [42]:
def generate_real_samples(dataset, batch_size):
    ix = np.random.randint(0,dataset.X.shape[0]-batch_size) #choose random index to start the sequence of n_samples
    X = dataset.X[ix:ix+batch_size]
    y = np.ones((batch_size,46,56,1)) #class lables is set to 1 for real samples
    return X, y

# Function to generate "Fake" samples from a defined latent space
def generate_latent_points(latent_dim, n_samples):
    #generate points in latent space
    x_input = np.random.randn(n_samples,latent_dim) #use "randn" for normal distribution and "rand" for uniform distribution
    x_input = x_input.reshape(n_samples, 46,56,16)
    return x_input

def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input)
    y = np.zeros((n_samples,46,56,1)) #class label is set to 0 for fake samples
    return X, y

In [67]:
epochs = 5
batch_size = 64
steps = 1

for i in range(0, epochs):
    #if (i%5 == 0):
    #    show_generator_results(generator_network)
    for j in range(steps):
        
        maps_input, maps_output = generate_real_samples(dg_train, 64)
        generated_maps_output = get_generated_samples(generator_network, maps_input)

        fake_patch = np.zeros((batch_size, 46, 56, 1))
        real_patch = np.ones((batch_size, 46, 56, 1))
        
        # Updating Discriminator weights
        discriminator_network.trainable=True
        loss_d1 = discriminator_network.train_on_batch(maps_input, real_patch)
        loss_d2 = discriminator_network.train_on_batch(generated_maps_output, fake_patch)
        
        loss_d = (np.add(loss_d1, loss_d2))/2.0
        
        
        maps_input, maps_output = generate_real_samples(dg_train, 64)
        
        # Make the Discriminator belive that these are real samples and calculate loss to train the generator
        real_patch = np.ones((batch_size, 46, 56, 1))
        
        # Updating Generator weights
        #discriminator_network.trainable=False
        #loss_g = dcgan_model.train_on_batch(maps_input, real_patch)

2022-06-29 14:01:38.528842: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled


In [43]:
n_batch=64
n_epochs=10
bat_per_epo = int(dataset.X.shape[0] / n_batch)
half_batch = int(n_batch / 2)
# manually enumerate epochs
for i in range(n_epochs):
    # enumerate batches over the training set
    for j in range(bat_per_epo):
        # get randomly selected 'real' samples
        X_real, y_real = generate_real_samples(dataset, half_batch)
        # update discriminator model weights
        d_loss1, _ = discriminator_network.train_on_batch(X_real, y_real)
        # generate 'fake' examples
        X_fake, y_fake = generate_fake_samples(generator_network, latent_dim, half_batch)
        # update discriminator model weights
        d_loss2, _ = discriminator_network.train_on_batch(X_fake, y_fake)
        # prepare points in latent space as input for the generator
        X_gan = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 46,56,1))
        # update the generator via the discriminator's error
        g_loss = dcgan_model.train_on_batch(X_gan, y_gan)
        # summarize loss on this batch
        print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
        (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
    

2022-06-29 14:11:36.605344: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201
2022-06-29 14:11:38.252701: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled


>1, 1/154, d1=9.172, d2=0.002 g=57.758


2022-06-29 14:11:39.617632: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled
2022-06-29 14:11:39.739376: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled


>1, 2/154, d1=5.547, d2=0.004 g=52.957


2022-06-29 14:11:39.929268: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled
2022-06-29 14:11:40.047283: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled


...

>10, 152/154, d1=0.000, d2=0.000 g=156.390


2022-06-29 14:19:54.531108: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled
2022-06-29 14:19:54.663327: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled


>10, 153/154, d1=0.000, d2=0.000 g=156.390


2022-06-29 14:19:54.872598: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled
2022-06-29 14:19:55.008904: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled


>10, 154/154, d1=0.000, d2=0.000 g=156.390


In [44]:
generator_network.save('tmp/test_nof/generator.h5')

In [45]:
X_g = generate_latent_points(latent_dim, 100)

In [48]:
predicted = generator_network.predict(X_g)